# Abstract words

In [1]:
# import all functions for this chapter/notebook
from abslithist.words import *

## Semantic fields

### Save word norms from psycholinguistic sources

These are downloaded from sources.

In [2]:
source(gen_orignorms)

def gen_orignorms():
    # init
    fields=defaultdict(set)
    norms=[]

    # add fields (so far, only quant/scale-based ones)
    funcs=[
        gen_norms_paivio, # Paivo et al
        gen_norms_mrc,    # MRC
        gen_norms_brys,   # Brysbaert et al
        gen_norms_lsn     # LSN
    ]

    # run through functions
    for func in tqdm(funcs,desc='Building fields and norms from sources'):
        func(norms)

    # save norms
    qdf=pd.DataFrame(norms)
    qdf=qdf.drop_duplicates(['word','source'],keep='first')
    qdf.to_csv(PATH_NORMS,index=False)

In [3]:
# gen_orignorms()

### Divide existing word norms into "fields"

This is done automatically by the z-score cutoff function in *get_fields_from_norms()*.

In [4]:
# Show the relevant functions
source(get_origfields)
source(get_fields_from_norms)

def get_origfields():
    dfnorms=pd.read_csv(PATH_NORMS)
    return get_fields_from_norms(dfnorms)

def get_fields_from_norms(dfnorms,zcut=ZCUT,name_neither='Neither',reverse=False,remove_stopwords=True):
    fields=dict()
    # if reverse: zcut=zcut*-1
    for source,sourcedf in dfnorms.groupby('source'):
        if not '.' in source: continue
        contrast,method=source.split('.')
        if not '-' in contrast: continue
        pos,neg=contrast.split('-')
        fields[f'{contrast}.{method}.{pos}']=poswords=set(sourcedf[sourcedf.z>=zcut if not reverse else sourcedf.z<=-zcut].word)
        fields[f'{contrast}.{method}.{neg}']=negwords=set(sourcedf[sourcedf.z<=-zcut if not reverse else sourcedf.z>=zcut].word)
        fields[f'{contrast}.{method}.{name_neither}'] = set(sourcedf.word) - poswords - negwords
    
    for field in fields:
        fields[field]={w for w in fields[field] if type(w)==str}

    if remove_stopwords:
        from abslithist.words import get_stopwords
        stopwords = get_stopwords()
        print(len(stopwords))
        for field in fields:
            print(f'removing {len(fields[field]&stopwords)} stopwords from {field}')
            fields[field]-=stopwords

    return fields

In [5]:
# Show fields
fields = get_origfields()
show_fields(fields)

/home/ryan/github/abslithist/data/fields/stopwords.txt
1307
removing 81 stopwords from Abs-Conc.LSN-Imag.Abs
removing 224 stopwords from Abs-Conc.LSN-Imag.Conc
removing 368 stopwords from Abs-Conc.LSN-Imag.Neither
removing 88 stopwords from Abs-Conc.LSN-Perc.Abs
removing 232 stopwords from Abs-Conc.LSN-Perc.Conc
removing 353 stopwords from Abs-Conc.LSN-Perc.Neither
removing 104 stopwords from Abs-Conc.LSN-Sens.Abs
removing 240 stopwords from Abs-Conc.LSN-Sens.Conc
removing 329 stopwords from Abs-Conc.LSN-Sens.Neither
removing 21 stopwords from Abs-Conc.MRC-Conc.Abs
removing 161 stopwords from Abs-Conc.MRC-Conc.Conc
removing 129 stopwords from Abs-Conc.MRC-Conc.Neither
removing 26 stopwords from Abs-Conc.MRC-Imag.Abs
removing 167 stopwords from Abs-Conc.MRC-Imag.Conc
removing 160 stopwords from Abs-Conc.MRC-Imag.Neither
removing 62 stopwords from Abs-Conc.MT-Conc.Abs
removing 222 stopwords from Abs-Conc.MT-Conc.Conc
removing 390 stopwords from Abs-Conc.MT-Conc.Neither
removing 3 stopwor

,num,words
field,,
Abs-Conc.LSN-Sens.Neither,24726,"winch, consultancy, newsroom, disorientate, incriminatory, bassoon, timetable, empathetic, nonun..."
Abs-Conc.LSN-Perc.Neither,24428,"hibernator, effervescent, couple, worried, shale, hilarious, abruptness, acquaint, worm, grieved..."
Abs-Conc.LSN-Imag.Neither,23683,"oyster, macabre, epic, handspring, lazily, demount, titleholder, teasing, circulator, agenda ......"
Abs-Conc.MT-Conc.Neither,22284,"funding, overkill, darer, grumpy, hacking, curtailment, disobey, disaster, picturesque, happier ..."
Abs-Conc.MT-Conc.Abs,7600,"pacifier, parkway, lime, eyewear, silkweed, modem, playbook, email, dashboard, license ... (7600)"
Abs-Conc.MT-Conc.Conc,6499,"presuppose, ego, unhesitating, unarguably, indelibly, edification, concisely, democracy, amateur..."
Abs-Conc.LSN-Imag.Abs,6444,"crypt, ostrich, impostor, cardboard, impenetrableness, stamp, unclogged, garment, wormwood, spin..."
Abs-Conc.LSN-Imag.Conc,6008,"indisposition, hyperirritability, counterclaim, infertility, formalize, sulfide, poeticize, iron..."
Abs-Conc.LSN-Perc.Abs,5868,"toenail, sob, scenery, feisty, grime, kaleidoscopic, harddisk, amputation, clutter, interrogate ..."


In [6]:
# Show contrasts
show_contrasts(fields)

neg  \
contrast method                                                        
Abs-Conc LSN-Imag  overture, exaltedness, spearmint, nonexclusive...   
         LSN-Perc  scarcity, barbarian, intangibility, immaterial...   
         LSN-Sens  presumptuously, contagiousness, undevelopable,...   
         MRC-Conc  considerable, compliment, literal, capacity, e...   
         MRC-Imag  unification, subaltern, unction, perturbation,...   
         MT-Conc   chummy, unhittable, sublimely, daringly, betra...   
         PAV-Conc  effort, supplication, dalliance, science, supp...   
         PAV-Imag  economy, savant, advantage, intellect, encepha...   

                                                                 pos  \
contrast method                                                        
Abs-Conc LSN-Imag  chairperson, wipeout, pantsuit, stonemason, sq...   
         LSN-Perc  panicky, overcook, messiness, lovemaking, dirt...   
         LSN-Sens  pooped, cheery, delightfully, lusty, skill, ra...   
         MRC-Conc  church, factory, desk, orange, tent, schoolhou...   
         MRC-Imag  cape, wall, sail, pickle, ruby, sand, coffin, ...   
         MT-Conc   atrium, chauffeur, jockey, sparkler, arugula, ...   
         PAV-Conc  ambulance, microscope, soil, dress, pencil, wh...   
         PAV-Imag  lip, pupil, bullet, frog, palace, cat, bouquet...   

                                                             neither  
contrast method                                                       
Abs-Conc LSN-Imag  spam, scribe, regime, indexed, distort, assume...  
         LSN-Perc  leading, gunfighter, gravitational, midpoint, ...  
         LSN-Sens  combatable, postmark, inflaming, ashy, postdoc...  
         MRC-Conc  sum, charity, bmost, blinks, wave, vaccination...  
         MRC-Imag  punish, czflora, clamor, nag, victory, pity, w...  
         MT-Conc   improving, petrify, rowing, disconnect, simult...  
         PAV-Conc  domicile, utensil, avenue, inhabitant, sky, te...  
         PAV-Imag  discovery, vessel, bravery, cost, background, ...

## Semantic vectors

### Compute vector-based word norms

At one and the same time: find the cosine distances between all the words of the model and the field vectors within that model; standardize the distances (z-score); and then save the standardized distances in a manner similar to how the original word norms were saved. Results are aggregated (median) first across runs of a model (e.g. BPO 1680-90), then across all models within larger period (C17, C18, etc).

In [7]:
source(gen_vecnorms)

def gen_vecnorms(bin_year_by=100):
    """
    Aggregate model-periods' vecnorms by century/yearbin
    """
    from abslithist.models.embeddings import get_model_paths

    def periodize(y):
        y=int(y)
        if bin_year_by==100:
            return f'C{(y//100) + 1}'
        else:
            return y//bin_year_by * bin_year_by

    # group by paths    
    paths_ld = get_model_paths()
    paths_df = pd.DataFrame(paths_ld)
    paths_df['period'] = paths_df['period_start'].apply(periodize)
    # print(len(paths_df))

    ## agg by period
    alldf=pd.DataFrame()
    for period,perioddf in sorted(paths_df.groupby('period')):
        # print(period,len(perioddf))
        # continue
        newdf=pd.DataFrame()
        for (corpus,period_start,period_end),cppdf in sorted(perioddf.groupby(['corpus','period_start','period_end'])):
            print(period,corpus,period_start,period_end)
            newdf=newdf.append(
                gen_vecnorms_for_paths(cppdf.path)
            )
            # break
        newdf=newdf.groupby(['word','source']).median().reset_index()
        newdf['period']=period
        alldf=alldf.append(newdf)
        # break
    alldf.to_csv(PATH_VECNORMS,index=False)

In [8]:
# Will take a long time!
# gen_vecnorms()

#### Get/inspect aggregated data

In [9]:
# Run?
# get_vecnorms()

### Divide vector-based norms into fields

In [10]:
vec_fields = get_vecfields()
# vec_fields

/home/ryan/github/abslithist/data/fields/stopwords.txt
1307
removing 172 stopwords from Abs-Conc.LSN-Imag.Abs
removing 133 stopwords from Abs-Conc.LSN-Imag.Conc
removing 660 stopwords from Abs-Conc.LSN-Imag.Neither
removing 184 stopwords from Abs-Conc.LSN-Perc.Abs
removing 200 stopwords from Abs-Conc.LSN-Perc.Conc
removing 581 stopwords from Abs-Conc.LSN-Perc.Neither
removing 172 stopwords from Abs-Conc.LSN-Sens.Abs
removing 157 stopwords from Abs-Conc.LSN-Sens.Conc
removing 636 stopwords from Abs-Conc.LSN-Sens.Neither
removing 152 stopwords from Abs-Conc.MRC-Conc.Abs
removing 173 stopwords from Abs-Conc.MRC-Conc.Conc
removing 640 stopwords from Abs-Conc.MRC-Conc.Neither
removing 158 stopwords from Abs-Conc.MRC-Imag.Abs
removing 137 stopwords from Abs-Conc.MRC-Imag.Conc
removing 670 stopwords from Abs-Conc.MRC-Imag.Neither
removing 155 stopwords from Abs-Conc.MT-Conc.Abs
removing 202 stopwords from Abs-Conc.MT-Conc.Conc
removing 608 stopwords from Abs-Conc.MT-Conc.Neither
removing 88 s

In [11]:
show_fields(vec_fields)

,num,words
field,,
Abs-Conc.LSN-Sens.Neither.C19,138721,"rescrve, muost, i'ope, prc1, 79s, .uu, 'jilt, dtscription, weith, fln ... (138721)"
Abs-Conc.LSN-Perc.Neither.C19,135500,"1vj, steplhen, contil, dt.., anxioui, irket, aqn, 1fy, loathing, dfj ... (135500)"
Abs-Conc.PAV-Conc.Neither.C19,135357,"heretofore, purts, t~'l, bch, 'lds, .att, kty, oiar, burmas, statvs ... (135357)"
Abs-Conc.MRC-Imag.Neither.C19,134497,"kiing, anjou, wlwn, '.ll, t~s, loinl, ii.tt, .airs, decease, jusqu'a ... (134497)"
Abs-Conc.LSN-Imag.Neither.C19,134441,"inequalities, bitbop, ijio, mutd, fsor, othcri, desolates, lhaving, pleae, inary ... (134441)"
...,...,...
Abs-Conc.LSN-Imag.Abs.C17,1134,"telling, into't, conicience, swear, taught, conic, shalt, itbe, loth, evident ... (1134)"
Abs-Conc.LSN-Sens.Abs.C17,1090,"proofs, lastly, glorious, signify, medals, scripture, dangers, andfor, explains, range ... (1090)"
Abs-Conc.PAV-Conc.Abs.C17,1074,"medium, evidences, propose, intention, parity, woulid, comnmon, understand, cafes, isno ... (1074)"


In [12]:
show_contrasts(vec_fields)

neg  \
contrast method                                                        
Abs-Conc LSN-Imag  kli, hcad, schola, .iid, folio, danceing, stan...   
         LSN-Imag  j., rai, jun, tlh, thtc, aint, wall, lop, ihi,...   
         LSN-Imag  56z, feconid, sluice, vtar, tr~e, iv,,, repass...   
         LSN-Imag  keys, pot, pull, pea, lily, sights, ingot, bus...   
         LSN-Perc  iss, antonini, rattle, '.r, oan, houi, grai, g...   
         LSN-Perc  blc, ody, lier, sits, diy, bre, rcs, C,, jq, ....   
         LSN-Perc  ranci, yok, istol, wiudow, lql, podge, wooing,...   
         LSN-Perc  trap, sand, pot, soup, leaf, wile, canyon, pin...   
         LSN-Sens  tjr, woolly, u,, mores, t:, doves, tone, .cr, ...   
         LSN-Sens  eb, ja, tnm, mile, boys, skill, fall, n., fall...   
         LSN-Sens  fwin, cheit, hou., awal, wlite, girths, heav, ...   
         LSN-Sens  coals, seizes, hurling, poise, wasps, ropes, f...   
         MRC-Conc  wearing, clare, durch, sweethings, lt., l7, ni...   
         MRC-Conc  woi, rle, george, ann, hio, atc, .cr, mac, ttb...   
         MRC-Conc  .ates, iris., forefoot, merck, 58s, 11o01, fun...   
         MRC-Conc  hogs, seeds, warming, rattled, volley, strips,...   
         MRC-Imag  southwest., dark, towvn, v*, hoa, aae, lks, de...   
         MRC-Imag  ifs, streets, i., kt, ttt, .nc, gardener, ihi,...   
         MRC-Imag  neur, gallops, nunnery, foiv, totur, mouth, il...   
         MRC-Imag  trapping, shovels, vegetation, midtown, blasti...   
         MT-Conc   threads, y6, llr, aiv, ftret, bucklers, w'r, ....   
         MT-Conc   .is, wt, 1y, chi, irs, tower, inf, ial, 'z, XV...   
         MT-Conc   rifs, divies, sope, sez, 2i3, gik, fevei, toiv...   
         MT-Conc   table, fires, youngster, mint, drawers, helper...   
         PAV-Conc  martins, pala, prier, 170o, hautboys, downing,...   
         PAV-Conc  'jc, wvit, ote, aio, advertise, vat, tonfon, d...   
         PAV-Conc  lkins, wd., etough, darl, efq., fountaine, mia...   
         PAV-Conc  finish, foot, wreath, sleeves, bake, softer, c...   
         PAV-Imag  vcl, 'js, bowl, wenr, nose, sounding, athis, s...   
         PAV-Imag  tbo, clark, fib, iun, dragon, b6, 01o, guildha...   
         PAV-Imag  vorn, peyton, thrash, thumping, sper, lusty, w...   
         PAV-Imag  boulder, boulevards, deftly, bat, hired, thumb...   

                                                                 pos  \
contrast method                                                        
Abs-Conc LSN-Imag  dots, covet, argues, repel, redouble, eftablil...   
         LSN-Imag  stronger, obedience, trxth, easy, slavery, ren...   
         LSN-Imag  'trust, xhat, revile, doobt, eforts, occafiou,...   
         LSN-Imag  realist, weaken, admits, revolting, unnatural,...   
         LSN-Perc  ordinats, undeniably, constant, praffical, abs...   
         LSN-Perc  endeavours, forsooth, names, levell, quietly, ...   
         LSN-Perc  habve, theological, territorial, ameliorate, m...   
         LSN-Perc  over-all, worthy, recommends, greater, underwr...   
         LSN-Sens  printing, dubious, fifthly, promote, tertullia...   
         LSN-Sens  events, fach, blame, abundantly, wholly, leen,...   
         LSN-Sens  alfumed, oj'the, lalande, jurists, incom, peop...   
         LSN-Sens  assistance, pacts, functionaries, considered, ...   
         MRC-Conc  affliction, patron, honeit, niore, emperours, ...   
         MRC-Conc  couch, vnited, operation, fooling, suppose, th...   
         MRC-Conc  unitarians, thabt, cevcry, deify, confequ, cos...   
         MRC-Conc  capitalistic, naught, uncertain, obstinacy, su...   
         MRC-Imag  shocks, agreements, endeavouring, mediate, pol...   
         MRC-Imag  plainer, afraid, supposition, deposing, sich, ...   
         MRC-Imag  dignity, ownership, deserves, monarchies, defe...   
         MRC-Imag  axiom, repression, illustrative, all-important...   
         MT-Conc   sufficiency, 